## Requests

In [11]:
import requests

In [12]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score
import numpy as np
import pandas as pd
import optuna
import json
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [13]:
X, y = make_classification(n_samples = 10000, 
                                             n_classes = 2, 
                                             n_features = 10, 
                                             n_informative = 10,
                                             n_redundant = 0,
                                             n_repeated = 0)
X = pd.DataFrame(X)
y = pd.Series(y)

In [14]:
train, test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    test_size=0.25)

In [16]:
train1 = json.dumps(train.to_json())
test1 = json.dumps(test.to_json())

In [17]:
# train.to_json('train.json')
# y_train.to_json('y_train.json')
# test.to_json('test.json')
# y_test.to_json('y_test.json')

**Вывод инфы:**

    1 - Проверить, что модель выдает список классов
    2 - Проверить, что модель выдает обученные модели 

In [157]:
resp = requests.get("http://localhost:5000/api/check_user", data = {'user_id': 11111})

In [158]:
resp.text

'"True"\n'

In [159]:
resp = requests.get("http://localhost:5000/api/ml_models/can_train")
eval(resp.text)

['logreg', 'forest', 'boosting']

In [160]:
resp = requests.get("http://localhost:5000/api/ml_models", data = {'user_id': 11111})
print(resp.text)

"[]"



In [161]:
resp = requests.get("http://localhost:5000/api/ml_models/count", data = {'user_id':11111})
eval(resp.text)

'0'

In [162]:
resp = requests.delete(
    "http://localhost:5000/api/ml_models", data={'user_id': 11111})
print(resp.text)

null



In [163]:
resp = requests.get("http://localhost:5000/api/ml_models/0", data = {'user_id':11111})
eval(resp.text)

'Only 0 models are trained!'

In [164]:
resp = requests.post("http://localhost:5000/api/ml_models/train", data = {'train': train1, 
                                                                          'y_train': y_train.to_json(), 
                                                                          'name': 'logreg',
                                                                          'user_id':11111})
eval(resp.text)

{'acc': 0.8653333333333334,
 'roc_auc': 0.9234999466666668,
 'aps': 0.9372601315533556}

In [165]:
resp = requests.post("http://localhost:5000/api/ml_models/train", data = {'train': train1, 
                                                                          'y_train': y_train.to_json(), 
                                                                          'test': test1, 
                                                                          'y_test': y_test.to_json(),
                                                                          'name': 'boosting',
                                                                          'user_id':11111})
eval(resp.text)

{'acc': 0.9552, 'roc_auc': 0.99052608, 'aps': 0.9911052487837202}

In [166]:
resp = requests.post("http://localhost:5000/api/ml_models/train", data = {'train': train1, 
                                                                          'y_train': y_train.to_json(), 
                                                                          'name': 'forest',
                                                                          'params': str({'max_depth':1}),
                                                                          'user_id':11111})
eval(resp.text)

{'acc': 0.7417333333333334, 'roc_auc': 0.85070976, 'aps': 0.8608221862544155}

In [167]:
resp = requests.post("http://localhost:5000/api/ml_models/train", data = {'train': train1, 
                                                                          'y_train': y_train.to_json(), 
                                                                          'name': 'forest',
                                                                          'find_params': True,
                                                                          'n_trials': 20,
                                                                          'cv':3,
                                                                          'user_id':11111})

In [168]:
eval(resp.text)

{'acc': 0.9118666666666666,
 'roc_auc': 0.9676170666666669,
 'aps': 0.9681748314634492}

In [172]:
resp = requests.get("http://localhost:5000/api/ml_models/predict/0", data = {'test': test1, 
                                                                             'user_id':11111})
res = eval(resp.text)

In [173]:
eval(res)

[0.1918870938600978,
 0.880157872933628,
 0.9952226979122447,
 0.272469521434471,
 0.026725854290648467,
 0.0482493486326333,
 0.016716004178579054,
 0.006198033481564062,
 0.5178980594439744,
 0.9029173585842223,
 0.9963227765797736,
 0.8742145758607986,
 0.4741465421159581,
 0.10668753930465777,
 0.0692169899344569,
 0.8179772081984571,
 0.15571805927778176,
 0.9203673104316645,
 0.3025211112962237,
 0.8410495167376026,
 0.07531258988515108,
 0.11157540186346422,
 0.19090771329834372,
 0.2548051284315547,
 0.1407014807594338,
 0.9917770554158032,
 0.12617221400813053,
 0.999567363933669,
 0.8884390694298301,
 0.1521122386433406,
 0.7569912907888667,
 0.06384915435149573,
 0.14841153991805434,
 0.4553181868617818,
 0.5496255406188201,
 0.09184059123071583,
 0.9013592898181627,
 0.6760517154112179,
 0.9725143808321357,
 0.9848930824488952,
 0.13780146649447744,
 0.6223768865355438,
 0.2569000925982634,
 0.970307365543986,
 0.009966600144941605,
 0.24873724670836203,
 0.9834242244591335